# Exploring cosine vs inner product.

This is a simple notebook to explore how various similarity metrics behave in the context of a collaborative filtering system where "ratings" come from implied feedback instead of direct feedback, e.g. users viewing and applying to job ads on a help wanted site.

First, import the recommendations code...

In [1]:
from pyrecommend import rec

This includes various utilities and similarity functions. Here's an example showing cosine similarity on a small dataset:

In [2]:
rec.sim(
    [1, 0, 1],
    [5, 0, 1],
    [0, 1, 0],
    [0, 1, 0],
    similarity=rec.similarity_cosine,
)

{(1, 2): 0.8320502943378437,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 4): 1.0}

Each list in the above dataset can be thought of as the rankings for an individual ad, where each index in thes lists represents a user. So in the above snippet, user 1 and user 3 both viewed ads 1 and 2. User 1 also applied to ad 2, since it has a score of 5 for him. Then, user 2 only viewed ads 3 and 4.

You can see that in this case that cosine similarity gives good results. 1 and 2 are similar, and 3 and 4 are similar, and that's all. And given the data, that makes perfect sense. Although curiously, 3 and 4 are rated as *more* similar than 1 and 2, likely due to the fact that the ratings between 1 and 2 weren't consistent. Perhaps counterintuitively, one user applying makes the two ads less similar.

Let's try again, making it so that user 1 applies to add 1 as well, to see if that will make them have 1.0 similarity.

In [3]:
rec.sim(
    [5, 0, 1],
    [5, 0, 1],
    [0, 1, 0],
    [0, 1, 0],
    similarity=rec.similarity_cosine,
)

{(1, 2): 1.0000000000000002,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 4): 1.0}

Floating point division is imperfect, but yes, it appears to be up to 1.0.

From now on rather than saying "user 1" and "user 2" I'm going to say "user A" and "user B" and so on.

## Where cosine becomes counterintuitive and gives poor results

So going back to the original dataset, does that mean if there was a third ad that users A and C both viewed but didn't apply to, it would be more similar to e.g. ad 1 than ad 2 is, with its single apply?

In [16]:
rec.sim(
    [1, 0, 1],
    [5, 0, 1],
    [1, 0, 1],  # the new ad
    [0, 1, 0],
    [0, 1, 0],
    similarity=rec.similarity_cosine,
)

{(1, 2): 0.8320502943378437,
 (1, 3): 0.9999999999999998,
 (1, 4): 0.0,
 (1, 5): 0.0,
 (2, 3): 0.8320502943378437,
 (2, 4): 0.0,
 (2, 5): 0.0,
 (3, 4): 0.0,
 (3, 5): 0.0,
 (4, 5): 1.0}

...These 0s are getting pretty noisy. Let's remove them...

In [20]:
def sim(*data, **kwargs):
    """Remove 0 records from similarity data."""
    similarity = kwargs.pop('similarity')
    if kwargs:
        raise ValueError('Unrecognized kwargs: {}'.format(kwargs.keys()))

    sim_data = rec.sim(*data, similarity=similarity).items()
    return {pair: score for pair, score in sim_data if score != 0}

def show_sim(*data, **kwargs):
    """Print data from sim() readably."""
    sim_data = sim(*data, **kwargs)
    for pair, score in sorted(sim_data.items()):
        print "{}: {}".format(pair, score)

In [6]:
show_sim(
    [1, 0, 1],
    [5, 0, 1],
    [1, 0, 1],  # the new ad
    [0, 1, 0],
    [0, 1, 0],
    similarity=rec.similarity_cosine,
)

(1, 2): 0.832050294338
(1, 3): 1.0
(2, 3): 0.832050294338
(4, 5): 1.0


That is a lot easier to deal with. So `(1, 3)` and `(4, 5)` have perfect similarity, which makes sense mathematically as they are identical. In a manual ratings system, where a user gives 1-5 stars, this may make more sense. But in our system where `1` means "viewed" and `5` means "applied," this doesn't seem ideal.

Let's try dot product on that dataset instead:

In [7]:
show_sim(
    [1, 0, 1],
    [5, 0, 1],
    [1, 0, 1],
    [0, 1, 0],
    [0, 1, 0],
    similarity=rec.dot_product,
)

(1, 2): 6
(1, 3): 2
(2, 3): 6
(4, 5): 1


So the most similar pairs are `(1, 2)` and `(2, 3)`. This seems a lot more ideal: If a user views ads 1 or 3, their highest recommendation will be 2. If a user views ad 2, they are suggested to both 1 and 3.

Another thing to note is that cosine similarity is normalized between 0 and 1, where dot product is unbounded. If same two ads are almost always applied to, one person not applying to it won't make a big difference, whereas in cosine similarity it may be more substantial. Let's try...

In [8]:
sim(
    
    # Two ads have tons of common applies, and one user who didn't apply.
    # The two uninterested users should not impact the scores.
    [5, 5, 5, 5, 5, 5, 0, 0],
    [5, 5, 5, 5, 5, 1, 1, 0],
    
    # A different island of ads, linked together by perhaps an accidental
    # view the G-th user made of the 2nd ad.
    [0, 0, 0, 0, 0, 0, 1, 5],
    [0, 0, 0, 0, 0, 0, 5, 1],

    similarity=rec.dot_product,
)

{(1, 2): 130, (2, 3): 1, (2, 4): 5, (3, 4): 10}

As expected, a user viewing 1 or 2 is directed to the other far and above the rest. Seems reasonable. A user viewing two will also get recommended to view 4, followed by 3. A user viewing ad 1 will not see those ads at all.

A user viewing ad 3 or 4 will be recommended the other, which seems ideal.

How does cosine deal with this dataset?

In [9]:
show_sim(
    [5, 5, 5, 5, 5, 5, 0, 0],
    [5, 5, 5, 5, 5, 1, 1, 0],
    
    [0, 0, 0, 0, 0, 0, 1, 5],
    [0, 0, 0, 0, 0, 0, 5, 1],

    similarity=rec.similarity_cosine,
)

(1, 2): 0.941880622803
(2, 3): 0.0174024929116
(2, 4): 0.0870124645582
(3, 4): 0.384615384615


Here the same exact story plays out. In fact, let's visualize the story using code.

In [10]:
def index_by_ad(dataset):
    """Build an ad-indexed view of similarity data.

    Each key is an ID, and each value is a list of (similarty_score, other_id) tuples,
    sorted so the most relevant match is first.

    """
    sim_scores = {}
    for pair, sim_score in dataset.items():
        ad_1, ad_2 = pair
        sim_scores.setdefault(ad_1, []).append((sim_score, ad_2))
        sim_scores.setdefault(ad_2, []).append((sim_score, ad_1))

    # Show most similar ads first
    for similar_ads in sim_scores.itervalues():
        similar_ads.sort(reverse=True)

    return sim_scores

With this we can now easily see what a viewer to an ad might see.

In [11]:
data = index_by_ad(sim(
    [5, 5, 5, 5, 5, 5, 0, 0],
    [5, 5, 5, 5, 5, 1, 1, 0],
    
    [0, 0, 0, 0, 0, 0, 1, 5],
    [0, 0, 0, 0, 0, 0, 5, 1],

    similarity=rec.similarity_cosine,
))

# What ads are similar to ad 2?
data[2]

[(0.9418806228028841, 1), (0.08701246455819893, 4), (0.017402492911639787, 3)]

So given the above users, a new user looking at ad 2 would be recommended 1, and then 4, and then 3. This makes a lot of sense. Ad 1 is by far the most similar to ad 2, whereas ad 4, while not similar, did get an apply from someone who viewed 2. Ad 3 is definitely the least suitable.

What do they get shown for ad 4? Here there is a ton more activity on ad 2, but it seems ad 3 is maybe more relevant overall. Given all the other activity on 2, user G is definitely an outlier by viewing it and not applying, then applying to unpopular ad 4 and viewing other unpopular ad 3. User H also seems to be more similar to G, and he applied to ad 3. So maybe people coming to ad 4 should be shown ad 3 instead of the far more popular ad 2?

In [12]:
data[4]

[(0.3846153846153847, 3), (0.08701246455819893, 2)]

Nice! Cosine similarity gives us seemingly good results. Ad 2 gets a low score, seemingly because only one user interested in 2 happened to be interested in 4 as well, where 6 other users of 2 didn't care for it at all. Let's try that again with dot product instead.

In [13]:
data = index_by_ad(sim(
    [5, 5, 5, 5, 5, 5, 0, 0],
    [5, 5, 5, 5, 5, 1, 1, 0],
    
    [0, 0, 0, 0, 0, 0, 1, 5],
    [0, 0, 0, 0, 0, 0, 5, 1],

    similarity=rec.dot_product,
))

data[4]

[(10, 3), (5, 2)]

So dot product puts them in the correct order, although ad 2 is much closer in score to ad 3 than it was with cosine. What does dot product give a user viewing ad 2?

In [14]:
data[2]

[(130, 1), (5, 4), (1, 3)]

Again, the order is correct.

## Conclusion

This was of course not exhaustive by any stretch. Cosine seems to prioritize similar usage behavior and will actually *derank* ads if a user applied amid a group of users who only viewed. This is the opposite of what we want. However there may be more to this story, I'm sure. I don't have time to dig into the details and exhaustively think of all cases, so for now I will simply choose dot product and see how it turns out.